# Prerequisites and Initial Setup

Most of the notebooks in the Azure SQL Hybrid Cloud Toolkit use either Microsoft PowerShell or Python for their script functionality. This notebook will perform the local machine configuration necessary to execute the various notebooks successfully.

## Microsoft PowerShell Version

Some notebooks use a [PowerShell Notebook kernel](https://youtu.be/BhbiAINQBYE) that requires PS v5.1 or greater.

In [ ]:
# Check that PowerShell >= v5.1 and Azure Command Line Interface (Az CLI) are installed
if ($PSVersionTable.PSVersion.Major -lt 5)
{
    Write-Warning "Some notebooks require a greater version of PowerShell."
}
else {
    $psVersion = $PSVersionTable.PSVersion.ToString()
    Write-Output "Found PS version... $psVersion"
}

## PowerShellGet and PowerShell Gallery Modules

If you do not have PS Nuget provider installed, then please see [Installing PowerShellGet](https://docs.microsoft.com/en-us/powershell/scripting/gallery/installing-psget). The initial setup for the PowerShellGet module may require elevated permissions and is recommended to be done in a separate PowerShell command prompt. 

The modules required by the toolkit are available on the [PowerShell Gallery](https://www.powershellgallery.com/), a central repository for sharing PS modules and scripts. The setup code in this notebook will create a temporary call to install required PowerShell modules in the current user scope. If the modules have not already been installed, the following code will install them with a new gallery registration setup as trusted to enable installation of modules without prompting. After installing the modules, the gallery registration will be removed.

| Module | Description |
| --- | --- |
| dbatools | Provides high-level cmdlets that perform SQL management operations. These CMDLETs have extensive validation tests and configuration options. |
| Az.Accounts | Provides cmdlets to configure subscription and account settings |
| Az.Resources | Provides cmdlets to access and provision Microsoft Azure resources |
| Az.Storage | Provides cmdlets to simplify configuration and access to Azure storage resources |
| Az.Network | Networking cmdlets for Azure Resource Manager |
| Az.Compute | Compute service cmdlets |
| AnyBox | Provides GUI input box support (Windows only) |
| SqlServer | Provides cmdlets to automate database development and server administration, as well as both multidimensional and tabular cube processing |

In [ ]:
# need to check if nuget package provider is installed
$nuget=(Get-PackageProvider -Name NuGet)

if (!$nuget)
{
    Get-PackageProvider -Name NuGet -ForceBootstrap
    Import-PackageProvider -Name NuGet -RequiredVersion 2.8.5.201
}

#temporary reposistory registration
Register-PSRepository -Name PSGalleryTemp -SourceLocation https://www.powershellgallery.com/api/v2/ -PublishLocation https://www.powershellgallery.com/api/v2/package/ -ScriptSourceLocation https://www.powershellgallery.com/api/v2/items/psscript/ -ScriptPublishLocation https://www.powershellgallery.com/api/v2/package/ -InstallationPolicy Trusted -PackageManagementProvider NuGet

#required modules for this notebook
$requiredModules = @("dbatools", "Az.Accounts", "Az.Resources", "Az.Storage", "Az.Network", "Az.Compute", "AnyBox","SqlServer")

#check each module and install if necessary
foreach ($moduleName in $requiredModules) {
    if (!(Get-Module -ListAvailable -Name $moduleName)) {
        Install-Module -Name $moduleName -Scope CurrentUser -AllowClobber -Repository PSGalleryTemp
    }
} 

#remove temporary gallery registration
Unregister-PSRepository -Name PSGalleryTemp

# Output any missing modules
foreach ($moduleName in $requiredModules) {
       $module = Get-InstalledModule $moduleName
    if (!($module)) {
          Write-Warning "The specified module is not installed...$moduleName"
    }
    else{
        $module
    }
}   

## SQL Assessment cmdlets and SMO extension

SQL Assessment API is part of the SQL Server Management Objects (SMO) and can be used with the SQL Server PowerShell module. Because installing the modules may require a local Administrator account's permission, it cannot be done automatically with this Notebook. The **Assessments** Notebooks require the following:

-   [Install SMO](https://docs.microsoft.com/en-us/sql/relational-databases/server-management-objects-smo/installing-smo?view=sql-server-ver15)
-   [Install SQL Server PowerShell module](https://docs.microsoft.com/en-us/sql/powershell/download-sql-server-ps-module?view=sql-server-ver15)

## Compatibility Assessment Tool - Data Migration Assistant

The Compatibility Assessment Notebook requires the Data Migration Assistant tool to be installed in order to execute. The installation link would be [Data Migration Assistant download](https://www.microsoft.com/en-us/download/confirmation.aspx?id=53595)

With version 2.1 and above, when installation of Data Migration Assistant is successful, it will install dmacmd.exe in _%ProgramFiles%\\Microsoft Data Migration Assistant_ folder.

## Azure CLI

Some notebooks also require the latest version of Azure command line interface (CLI). To install, see [https://aka.ms/installazurecliwindows](https://aka.ms/installazurecliwindows). Update instructions can be found at [https://aka.ms/doc/UpdateAzureCliMsi](https://aka.ms/doc/UpdateAzureCliMsi)

In [ ]:
# Check for Azure CLI
$azCli = (az --version)
if ($azCli) {        
    $azVersion = $azCli[0].Split()[-1]
    if ($azVersion -eq "*") {
        $azVersion = $azCli[0].Split()[-2]
    }
    Write-Output "Found Az CLI version... $azVersion"
}
else {
    Write-Warning "Some notebooks require the Az CLI. Please install if to run these notebooks."
}

## PowerShell ADS Extension

It may also be useful to install the official release of the PowerShell ADS extension by following the steps in the [Azure Data Studio documentation](vscode-webview://extensioneditor/electron-browser/docs.microsoft.com/en-us/sql/azure-data-studio/extensions). Installing this extension is optional but will enable syntax highlighting and intellisense within ADS. 

In the Extensions pane, search for "PowerShell" extension and install it there. It is also possible to install a VSIX package from the [PowerShell extension's Releases page](https://github.com/PowerShell/vscode-powershell/releases) through the command line:

`azuredatastudio --install-extension PowerShell-<version>.vsix`

In [ ]:
# azuredatastudio --install-extension PowerShell-v2020.9.0-preview.vsix     # Uncomment to install a specific version